<a href="https://colab.research.google.com/github/rpmullig/Coding_Math/blob/master/CIFAR_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

# Confirm that we're using Python 3
assert sys.version_info.major is 3, 'Oops, not running Python 3. Use Runtime > Change runtime type'

print("Installing dependencies for Colab environment")
!pip install -Uq grpcio==1.26.0

import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print('TensorFlow version: {}'.format(tf.__version__))


cifar100 = tf.keras.datasets.cifar100

(train_images, train_labels), (test_images, test_labels) = cifar100.load_data(label_mode='fine')

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 32, 32, 3)
test_images = test_images.reshape(test_images.shape[0], 32, 32, 3)

print('\ntrain_images.shape: {}, of {}'.format(train_images.shape, train_images.dtype))
print('test_images.shape: {}, of {}'.format(test_images.shape, test_images.dtype))

Installing dependencies for Colab environment
TensorFlow version: 2.2.0

train_images.shape: (50000, 32, 32, 3), of float64
test_images.shape: (10000, 32, 32, 3), of float64


In [21]:
model = keras.Sequential([
  keras.layers.Conv2D(128, kernel_size=(3,3), input_shape=(32,32,3), activation='elu', name='Conv1'),
  keras.layers.Conv2D(128, kernel_size=(3,3), activation='elu', name='Conv2'),
  keras.layers.MaxPool2D(pool_size=(2,2)),
  keras.layers.Dropout(0.1),

  keras.layers.Conv2D(256, kernel_size=(3,3), activation='elu', name='Conv3'),
  keras.layers.Conv2D(256, kernel_size=(3,3), activation='elu', name='Conv4'),
  keras.layers.MaxPool2D(pool_size=(2,2)),
  keras.layers.Dropout(0.25),

  keras.layers.Flatten(), 

  keras.layers.Dense(1024, activation='elu'),
  keras.layers.Dropout(0.5),
  keras.layers.Dense(100, activation=tf.nn.softmax, name='Softmax')
])
model.summary()

testing = False
epochs = 15

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],)

model.fit(train_images, train_labels, epochs=epochs)

test_loss, test_acc = model.evaluate(test_images, test_labels,)
print('\nTest accuracy: {}'.format(test_acc))

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv2D)               (None, 30, 30, 128)       3584      
_________________________________________________________________
Conv2 (Conv2D)               (None, 28, 28, 128)       147584    
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 14, 14, 128)       0         
_________________________________________________________________
dropout_55 (Dropout)         (None, 14, 14, 128)       0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 12, 12, 256)       295168    
_________________________________________________________________
Conv4 (Conv2D)               (None, 10, 10, 256)       590080    
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 5, 5, 256)       

In [22]:
import numpy as np 

test_data = np.load("cifar100_test.npy")

test_data = test_data / 255.0


test_data = test_data.reshape(test_data.shape[0], 32, 32, 3)


print('test_data.shape: {}, of {}'.format(test_data.shape, test_data.dtype))

predictions = model.predict(test_data)

output = np.array(np.argmax(predictions[0]), dtype=np.float64)

for i in range(1, predictions.shape[0]):
  output = np.append(output, np.argmax(predictions[i]))


import pandas as pd
from google.colab import files

df = pd.DataFrame(output, columns=["Category"])
df.index.name = "Id"
df.to_csv('predictions_1.csv')
files.download('predictions_1.csv')

test_data.shape: (10000, 32, 32, 3), of float64
